# OPTaaS Quick Start

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. If you don't have one, <a href="mailto:optaas@mindfoundry.ai">email us to start your free trial</a>.

More tutorials are [available here](/notebooks)

## Connect to OPTaaS using your API Key

In [1]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Define your parameters

In [2]:
from mindfoundry.optaas.client.parameter import IntParameter, FloatParameter, CategoricalParameter, BoolParameter, \
    ChoiceParameter, GroupParameter

bool_param = BoolParameter('my_bool')
cat_param = CategoricalParameter('my_cat', values=['a', 'b', 'c'], default='c')

int_param = IntParameter('my_int', minimum=0, maximum=20)
optional_int_param = IntParameter('my_optional_int', minimum=-10, maximum=10, optional=True)

parameters = [
    bool_param,
    cat_param,
    ChoiceParameter('ints_or_floats', choices=[
        GroupParameter('ints', items=[int_param, optional_int_param]),
        GroupParameter('floats', items=[
            FloatParameter('float1', minimum=0, maximum=1),
            FloatParameter('float2', minimum=0.5, maximum=4.5)
        ])
    ]),
]

## Define your scoring function

**Note:** the argument names in your scoring function must match the parameter names you defined above.

In [3]:
def scoring_function(my_bool, my_cat, ints_or_floats):
    score = 5 if my_bool is True else -5
    score += 1 if my_cat == 'a' else 3
    if 'ints' in ints_or_floats:
        score += sum(ints_or_floats['ints'].values())
    else:
        score *= sum(ints_or_floats['floats'].values())
    return score

## Create your Task

You can use Goal.max or Goal.min as appropriate.

In [4]:
from mindfoundry.optaas.client.client import Goal

task = client.create_task(
    title='Quick Start Example Task',
    parameters=parameters,
    goal=Goal.max,
)

## Run your Task

We will run for a maximum of 50 iterations, but we will stop as soon as we reach our score threshold of 32.

The score threshold is optional - you can omit it and simply run as many iterations as you need.

In [5]:
best_result, best_configuration = task.run(scoring_function, max_iterations=50, score_threshold=32)
print(f'Best score: {best_result.score}')
print(f'Best configuration: {best_configuration.values}')

Running task "Quick Start Example Task" for 50 iterations
(or until score is 32 or better)

Iteration: 0    Score: -6.0
Configuration: {'my_bool': False, 'my_cat': 'c', 'ints_or_floats': {'floats': {'float1': 0.5, 'float2': 2.5}}}

Iteration: 1    Score: 17
Configuration: {'my_bool': True, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 9}}}

Iteration: 2    Score: -10.008750866716957
Configuration: {'my_bool': False, 'my_cat': 'a', 'ints_or_floats': {'floats': {'float1': 0.050884602994662465, 'float2': 2.4513031136845767}}}

Iteration: 3    Score: 10
Configuration: {'my_bool': True, 'my_cat': 'a', 'ints_or_floats': {'ints': {'my_int': 3, 'my_optional_int': 1}}}

Iteration: 4    Score: 20
Configuration: {'my_bool': True, 'my_cat': 'b', 'ints_or_floats': {'ints': {'my_int': 12}}}

Iteration: 5    Score: 1
Configuration: {'my_bool': False, 'my_cat': 'b', 'ints_or_floats': {'ints': {'my_int': 1, 'my_optional_int': 2}}}

Iteration: 6    Score: 13
Configuration: {'my_bool': True, 'my_c